In [1]:
import Pkg
using Distributed
addprocs(5)
using XLSX

In [6]:
@everywhere include("plans_coupants.jl")

In [8]:
end_time=10
futures_planscoupants = Vector{Future}()

for name_instance in readdir("./data/")[2: 7]
    push!(futures_planscoupants, @spawn plans_coupantsALG(name_instance, "no_symmetry", "with initial values", end_time))
end

results = map(fetch, futures_planscoupants)

6-element Vector{Tuple{String, Float64, Vector{Int64}, Float64, Float64, Int64, Int64}}:
 ("1000_USA-road-d.BAY.gr", 10.476999998092651, [205, 216, 218, 299, 335, 454, 516, 560, 642, 656, 659, 716, 856, 915, 964, 965, 993], 48250.00000000001, 40117.0, 0, 2)
 ("1000_USA-road-d.COL.gr", 10.452999830245972, [18, 27, 117, 144, 212, 213, 319, 407, 546, 576, 583, 666, 741, 779], 29904.819999999996, 24068.0, 0, 2)
 ("1000_USA-road-d.NY.gr", 10.333999872207642, [50, 54, 85, 120, 256, 281, 323, 330, 349, 359, 360, 384, 396, 454, 576, 621, 640, 759, 906], 43841.310000000005, 36881.636, 0, 2)
 ("100_USA-road-d.BAY.gr", 0.6649999618530273, [4, 10, 44, 59, 65, 74, 89], 10857.060000000001, 10857.060000000001, 1, 2)
 ("100_USA-road-d.COL.gr", 0.21399998664855957, [7, 26, 49, 59, 71, 77, 79, 81, 83, 92, 93, 97], 25320.15, 25320.15, 1, 5)
 ("100_USA-road-d.NY.gr", 2.0889999866485596, [7, 15, 16, 42, 51, 54, 55, 68, 96], 33931.04, 33931.04, 1, 4)

In [54]:
XLSX.openxlsx("results_planscoupants.xlsx", mode="w") do xf
    sheet = xf[1]
    XLSX.rename!(sheet, "results")
    sheet["A1"]="Instance"
    sheet["B1"]="Temps"
    sheet["C1"]="Solution"
    sheet["D1"]="Meilleure solution"
    sheet["E1"]="Borne inf"
    sheet["F1"]="Coupes 23"
    sheet["G1"]="Coupes 24"
    for i in 1:length(results)
        sheet["A"*string(i+1)]= results[i][1]
        sheet["B"*string(i+1)]= results[i][2]
        sheet["C"*string(i+1)]= join(string.(results[i][3]), ", ")
        sheet["D"*string(i+1)]= results[i][4]
        sheet["E"*string(i+1)]= results[i][5]
        sheet["F"*string(i+1)]= results[i][6]
        sheet["G"*string(i+1)]= results[i][7]
    end
end